In [1]:
from dart_fss import get_corp_list
import dart_fss as dart
import pandas as pd
import requests

# 1. API 키 설정
import os
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경변수에서 API 키 가져오기
API_KEY = os.getenv("API_KEY")

dart.set_api_key(api_key=API_KEY)

# 모든 상장된 기업 리스트 불러오기
corp_list = get_corp_list()


# 삼성전자를 이름으로 찾기 ( 리스트 반환 )
samsung = corp_list.find_by_corp_name('삼성전자', exactly=True)[0]
samsung

Error occurred during getting browser(s): random, but was suppressed with fallback.


Label,Data
corp_code,00126380
corp_name,삼성전자
corp_eng_name,"SAMSUNG ELECTRONICS CO,.LTD"
stock_code,005930
modify_date,20250326
sector,통신 및 방송 장비 제조업
product,"통신 및 방송 장비 제조(무선) 제품, 반도체 제조(메모리) 제품, 전자부품 제조(디스플레이) 제품, 영상 및 음향기기 제조(영상기기) 제품 등"
corp_cls,Y


In [2]:
corp_list.find_by_corp_name('삼성전자')
# corp_name = ["삼성전자", "LG전자", "에코프로", "현대중공업", "현대자동차"]
corp_name = ["삼성전자"]
for corp in corp_name:
    name = corp_list.find_by_corp_name(corp, exactly=True)
    if name:
        number = str(name[0])[1:9]
    else:
        continue
    print(number)

00126380


In [4]:
print(name[0])

[00164742]현대자동차


In [ ]:
year = "2024"
reprt = "11011"
url = f"https://opendart.fss.or.kr/api/fnlttMultiAcnt.json?crtfc_key={api_key}&corp_code={number}&bsns_year={year}&reprt_code={reprt}"

In [29]:
import json
# --- API 요청 URL 및 파라미터 ---
base_url = "https://opendart.fss.or.kr/api/fnlttMultiAcnt.json"
params = {
    'crtfc_key': api_key,
    'corp_code': number,
    'bsns_year': year,
    'reprt_code': reprt,
}

# --- GET 요청 보내기 ---
try:
    response = requests.get(base_url, params=params)
    
    # --- 응답 확인 ---
    # HTTP 상태 코드가 200 (OK)인지 확인
    if response.status_code == 200:
        # XML 형식의 응답 내용을 출력
        print("요청 성공!")
        print(response.text) 
    else:
        print(f"에러 발생: HTTP 상태 코드 {response.status_code}")
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"요청 중 에러 발생: {e}")

요청 성공!
{"status":"000","message":"정상","list":[{"rcept_no":"20250311001085","reprt_code":"11011","bsns_year":"2024","corp_code":"00126380","stock_code":"005930","fs_div":"CFS","fs_nm":"연결재무제표","sj_div":"BS","sj_nm":"재무상태표","account_nm":"유동자산","thstrm_nm":"제 56 기","thstrm_dt":"2024.12.31 현재","thstrm_amount":"227,062,266,000,000","frmtrm_nm":"제 55 기","frmtrm_dt":"2023.12.31 현재","frmtrm_amount":"195,936,557,000,000","bfefrmtrm_nm":"제 54 기","bfefrmtrm_dt":"2022.12.31 현재","bfefrmtrm_amount":"218,470,581,000,000","ord":"1","currency":"KRW"},{"rcept_no":"20250311001085","reprt_code":"11011","bsns_year":"2024","corp_code":"00126380","stock_code":"005930","fs_div":"CFS","fs_nm":"연결재무제표","sj_div":"BS","sj_nm":"재무상태표","account_nm":"비유동자산","thstrm_nm":"제 56 기","thstrm_dt":"2024.12.31 현재","thstrm_amount":"287,469,682,000,000","frmtrm_nm":"제 55 기","frmtrm_dt":"2023.12.31 현재","frmtrm_amount":"259,969,423,000,000","bfefrmtrm_nm":"제 54 기","bfefrmtrm_dt":"2022.12.31 현재","bfefrmtrm_amount":"229,953,926,00

In [32]:
data = json.loads(response.text)

# 조건: account_nm == '유동자산'
target_account_nm = "유동자산"
results = []

for item in data.get("list", []):
    if item.get("account_nm") == target_account_nm:
        print(f"[{target_account_nm}]")
        print(f"- {item.get('fs_div')}")
        print(f"- {item.get('thstrm_nm')} ({item.get('thstrm_dt')}): {item.get('thstrm_amount')} (당기)")
        print(f"- {item.get('frmtrm_nm')} ({item.get('frmtrm_dt')}): {item.get('frmtrm_amount')} (전기)")
        if item.get("bfefrmtrm_amount"):
            print(f"- {item.get('bfefrmtrm_nm')} ({item.get('bfefrmtrm_dt')}): {item.get('bfefrmtrm_amount')} (전전기)")

[유동자산]
- CFS
- 제 56 기 (2024.12.31 현재): 227,062,266,000,000 (당기)
- 제 55 기 (2023.12.31 현재): 195,936,557,000,000 (전기)
- 제 54 기 (2022.12.31 현재): 218,470,581,000,000 (전전기)
[유동자산]
- OFS
- 제 56 기 (2024.12.31 현재): 82,320,322,000,000 (당기)
- 제 55 기 (2023.12.31 현재): 68,548,442,000,000 (전기)
- 제 54 기 (2022.12.31 현재): 59,062,658,000,000 (전전기)


In [34]:
target_accounts = ["유동자산", "비유동자산", "자산총계", "부채총계"]

# 저장할 데이터를 리스트 형태로 정리
rows = []

for item in data.get("list", []):
    account_nm = item.get("account_nm")
    if account_nm in target_accounts:
        row = {
            "계정명": account_nm,
            "당기명": item.get("thstrm_nm"),
            "당기일자": item.get("thstrm_dt"),
            "당기금액": item.get("thstrm_amount"),
            "전기명": item.get("frmtrm_nm"),
            "전기일자": item.get("frmtrm_dt"),
            "전기금액": item.get("frmtrm_amount"),
            "전전기명": item.get("bfefrmtrm_nm"),
            "전전기일자": item.get("bfefrmtrm_dt"),
            "전전기금액": item.get("bfefrmtrm_amount"),
            "공시번호": item.get("rcept_no"),
            "재무제표종류": item.get("fs_nm"),
            "재무제표명": item.get("sj_nm")
        }
        rows.append(row)

# DataFrame 생성
df = pd.DataFrame(rows)

# CSV 저장
df.to_csv("계정별_재무데이터.csv", index=False, encoding='utf-8-sig')